In [21]:

import pandas as pd
import geopandas as gpd
import xgboost
import sys
from sklearn.preprocessing import LabelEncoder
import numpy as np
from math import * 
from tqdm import tqdm
from sklearn import model_selection
from sklearn.metrics import roc_auc_score
from sklearn import preprocessing
from sklearn.metrics import classification_report
from sklearn.model_selection import train_test_split
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import cross_val_score
from sklearn.metrics import accuracy_score
from sklearn.ensemble import RandomForestClassifier

from sklearn.cluster import DBSCAN, OPTICS

import json
import matplotlib.pyplot as plt
import matplotlib as mpl
mpl.rcParams['figure.dpi'] = 250
plt.style.use('dark_background')

from utils import plot_cluster, load_list, save_list

In [22]:
n_data_max = 999999999

In [23]:
train_df = pd.read_csv('train_df.csv')[:n_data_max]
test_df = pd.read_csv('test_df.csv')[:n_data_max]

In [24]:
train_df_origin = gpd.read_file("train.geojson", index_col=0)[:n_data_max]
test_df_origin = gpd.read_file("test.geojson", index_col=0)[:n_data_max]

In [25]:
#Keys are columns that will be augmented. Values tell us whether they are numerical or not (categorical).
cols_is_num = {'area':True,
            'length':True, 
            'area/length**2':True,
            'elongation':True, 
            'centroid_x':True,      #Irrelevant, let to see if augmentation works (knn_centroid_x should be about same value as centroid_x)
            'height':True, 
            'width':True, 
            'nb_points':True, 
            'centroid_dist':True, 
            'length/width':True, 
            
            'Dense Urban':True, 
            'Industrial':True, 
            'None':True,
            'Rural':True, 
            'Sparse Urban':True, 
            'Urban Slum':True, 
            'Barren Land':True, 
            'Coastal':True,
            'Dense Forest':True, 
            'Desert':True, 
            'Farms':True, 
            'Grass Land':True, 
            'Hills':True, 
            'Lakes':True,
            'None.1':True, 
            'River':True, 
            'Snow':True, 
            'Sparse Forest':True,
            
            'diff1':True, 
            'diff2':True, 
            'diff3':True,
            'diff4':True,
            'year_date1':True, 
            'year_date2':True, 
            'year_date3':True, 
            'year_date4':True,
            'year_date5':True,
            'season_date1':True, 
            'season_date2':True, 
            'season_date3':True, 
            'season_date4':True,
            'season_date5':True,
            }

In [26]:
from sklearn.neighbors import NearestNeighbors

def find_knn(i, knn_object, df_aug):
    '''Return a dataframe composed of the k nearest neighbors in df_aug of the i-th data of df_aug.
    i: index of data
    knn_object : a NearestNeighbors object fitted on df_aug
    df_aug : dataframe augmented
    '''
    data = np.array(df_aug[['centroid_x', 'centroid_y']].iloc[i])
    distances, indices = knn_object.kneighbors(data.reshape(-1,2), return_distance = True)      #We return the k+1 nearest neighbors, except the first (the data point himself)
    return df_aug.iloc[indices[0, 1:]], distances[0, 1:]
    

def nearest_buildings_augmentation_mean(df_aug, k = 10, cols_is_num = cols_is_num, method = 'mean'):
    '''Return a dataframe whose columns are mean or most frequent value of k nearest neighbors features. The columns thus augmented are those in 'cols_is_num'.
    df_aug : an already augmented dataframe
    k : the number of nearest neighbors
    cols : a dictionnary whose keys are name of columns to be copied and values are True if feature is numerical, False if categorical.
    '''
    df = list()
    knn_object = NearestNeighbors(n_neighbors=k+1).fit(np.array(df_aug[['centroid_x', 'centroid_y']]))
        
    for i in tqdm(range(len(df_aug))):
        dic = dict()
        df_k_nearest_neightbors, distances = find_knn(i, knn_object, df_aug)      #Find the k nearest neighbors.
        
        if method == 'weighted':
            m = sum(distances)/k
            poids = [exp(-x/m) for x in distances]
            poids = [x/sum(poids) for x in poids]
            for col, is_num in cols_is_num.items():
                if is_num:
                    dic[f"knn_mean_{col}"] = (df_k_nearest_neightbors[col] * poids).sum()
                else:
                    raise
        elif method == 'mean':
            for col, is_num in cols_is_num.items():
                if is_num:
                    dic[f"knn_mean_{col}"] = df_k_nearest_neightbors[col].mean()              
                else:
                    raise
        df.append(dic)
    return pd.DataFrame(df)

k = 15
df = nearest_buildings_augmentation_mean(train_df, k = k, method = 'mean')
df.to_csv('train_df_knn_mean.csv')
df.head()


100%|██████████| 309736/309736 [44:26<00:00, 116.18it/s]


In [ ]:
def nearest_buildings_augmentation_concatenate(df_aug, k = 1, cols = cols_is_num):
    '''Return a dataframe whose columns are features of nearest neighbors. If cols contains n feature names, this will return a n*k feature dataframe.
    df_aug : an already augmented dataframe
    k : the number of nearest neighbors
    cols : a dictionnary whose keys are name of columns to be copied
    '''
    df = list()
    data = np.array(df_aug[['centroid_x', 'centroid_y']])
    knn_object = NearestNeighbors(n_neighbors=k+1).fit(data)     #knn object to find neighbors
        
    for i in tqdm(range(len(df_aug))):
        dic = dict()
        data = np.array(df_aug[['centroid_x', 'centroid_y']].iloc[i])
        indices = knn_object.kneighbors(data.reshape(-1,2), return_distance = False)[0, 1:] 
        df_knn = df_aug.iloc[indices]
        
        for col in cols.keys():
            feature = df_knn[col]
            for n in range(k):
                dic[f"{n+1}th_nn_{col}"] = feature.iloc[n]                  
                
        df.append(dic)
    return pd.DataFrame(df)

df = nearest_buildings_augmentation_concatenate(train_df, k = 1)
df.to_csv('train_df_knn_concat.csv')
df.head()

100%|██████████| 50000/50000 [03:06<00:00, 267.83it/s]


,1th_nn_area,1th_nn_length,1th_nn_area/length**2,1th_nn_elongation,1th_nn_centroid_x,1th_nn_height,1th_nn_width,1th_nn_nb_points,1th_nn_centroid_dist,1th_nn_length/width,...,1th_nn_year_date1,1th_nn_year_date2,1th_nn_year_date3,1th_nn_year_date4,1th_nn_year_date5,1th_nn_season_date1,1th_nn_season_date2,1th_nn_season_date3,1th_nn_season_date4,1th_nn_season_date5
0,4.701495e-07,0.002917,0.055257,0.001024,116.975303,0.001024,0.000603,5,0.002188,1.956201,...,2014,2015,2017,2018,2020,3,3,3,3,3
1,1.237159e-06,0.005545,0.040231,0.002244,116.976067,0.002244,0.000888,6,0.004642,3.847414,...,2014,2015,2017,2018,2020,3,3,3,3,3
2,1.237159e-06,0.005545,0.040231,0.002244,116.976067,0.002244,0.000888,6,0.004642,3.847414,...,2014,2015,2017,2018,2020,3,3,3,3,3
3,3.516945e-07,0.002763,0.046052,0.001110,116.976916,0.000547,0.001110,5,0.002232,3.034208,...,2014,2015,2017,2018,2020,3,3,3,3,3
4,3.655977e-07,0.002749,0.048380,0.001063,116.978161,0.000547,0.001063,5,0.002182,2.661171,...,2014,2015,2017,2018,2020,3,3,3,3,3


In [ ]:
df = nearest_buildings_augmentation_mean(test_df, k = k)
df.to_csv('test_df_knn_mean.csv')
df.head()

In [ ]:
df = nearest_buildings_augmentation_concatenate(test_df, k = 1)
df.to_csv('test_df_knn_concat.csv')
df.head()

100%|██████████| 1000/1000 [00:02<00:00, 370.78it/s]


,1th_nn_area,1th_nn_length,1th_nn_area/length**2,1th_nn_elongation,1th_nn_centroid_x,1th_nn_height,1th_nn_width,1th_nn_nb_points,1th_nn_centroid_dist,1th_nn_length/width,...,1th_nn_year_date1,1th_nn_year_date2,1th_nn_year_date3,1th_nn_year_date4,1th_nn_year_date5,1th_nn_season_date1,1th_nn_season_date2,1th_nn_season_date3,1th_nn_season_date4,1th_nn_season_date5
0,8.068710e-08,0.001467,0.037502,0.000625,103.974741,0.000275,0.000625,5,0.001250,4.434898,...,2014,2015,2017,2018,2020,3,3,3,3,3
1,4.833666e-08,0.001253,0.030788,0.000531,103.975182,0.000236,0.000531,5,0.001116,5.037163,...,2014,2015,2017,2018,2020,3,3,3,3,3
2,1.425909e-07,0.001997,0.035759,0.000838,103.975004,0.000385,0.000838,5,0.001711,4.279575,...,2014,2015,2017,2018,2020,3,3,3,3,3
3,7.172681e-08,0.001103,0.059005,0.000362,103.974644,0.000360,0.000362,5,0.000852,1.455363,...,2014,2015,2017,2018,2020,3,3,3,3,3
4,8.068710e-08,0.001467,0.037502,0.000625,103.974741,0.000275,0.000625,5,0.001250,4.434898,...,2014,2015,2017,2018,2020,3,3,3,3,3
